# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments  \
0  Jun 29 2022  8:21AM  240686        10   0085811558   
1  Jun 29 2022  8:21AM  240686        10   0085811860   
2  Jun 29 2022  8:21AM  240686        10   0085811919   
3  Jun 29 2022  8:19AM  240688        10   0085811897   
4  Jun 29 2022  8:19AM  240688        10   0085811828   
5  Jun 29 2022  8:19AM  240688        10   0085811769   
6  Jun 29 2022  8:19AM  240688        10   0085811775   
7  Jun 29 2022  8:19AM  240688        10   0085811820   
8  Jun 29 2022  8:19AM  240688        10   0085811825   
9  Jun 29 2022  8:14AM  240687        16  SAR00000347   

                           Customer ShipmentStatus  
0                 ISDIN Corporation       Released  
1                 ISDIN Corporation       Released  
2                 ISDIN Corporation       Released  
3                 ISDIN Corporation       Released  
4                 ISDIN Corporation       Released  
5                 ISDIN Corporation       Released  
6                 ISDIN Corporation       Released  
7                 ISDIN Corporation       Released  
8                 ISDIN Corporation       Released  
9  Sartorius Stedim North America        Executing

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
23  240684       Released          3
24  240685       Released          6
25  240686       Released          3
26  240687      Executing          1
27  240688       Released          6

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
240684                NaN        NaN       3.0
240685                NaN        NaN       6.0
240686                NaN        NaN       3.0
240687                NaN        1.0       NaN
240688                NaN        NaN       6.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
240599                7.0        1.0      13.0
240613                0.0        0.0       1.0
240623                0.0        0.0       1.0
240624                0.0        0.0       1.0
240629                0.0        1.0       0.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
240599                  7          1        13
240613                  0          0         1
240623                  0          0         1
240624                  0          0         1
240629                  0          1         0

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               240599          7          1        13
1               240613          0          0         1
2               240623          0          0         1
3               240624          0          0         1
4               240629          0          1         0

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               240599         7         1       13
1               240613                            1
2               240623                            1
3               240624                            1
4               240629                   1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                          Customer
0   Jun 29 2022  8:21AM  240686        10                 ISDIN Corporation
3   Jun 29 2022  8:19AM  240688        10                 ISDIN Corporation
9   Jun 29 2022  8:14AM  240687        16  Sartorius Stedim North America  
10  Jun 29 2022  8:02AM  240685        10                 ISDIN Corporation
16  Jun 29 2022  8:02AM  240684        10                 ISDIN Corporation
19  Jun 28 2022  4:38PM  240682        12  Twinlab Consolidated Corporation
20  Jun 28 2022  4:31PM  240680        12                         Hush Hush
22  Jun 28 2022  4:14PM  240679        21                 NBTY Global, Inc.
23  Jun 28 2022  3:32PM  240672        19               AdvaGen Pharma, Ltd
55  Jun 28 2022  3:31PM  240673        20  Sartorius Stedim North America

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                          Customer  \
0  Jun 29 2022  8:21AM  240686        10                 ISDIN Corporation   
1  Jun 29 2022  8:19AM  240688        10                 ISDIN Corporation   
2  Jun 29 2022  8:14AM  240687        16  Sartorius Stedim North America     
3  Jun 29 2022  8:02AM  240685        10                 ISDIN Corporation   
4  Jun 29 2022  8:02AM  240684        10                 ISDIN Corporation   
5  Jun 28 2022  4:38PM  240682        12  Twinlab Consolidated Corporation   
6  Jun 28 2022  4:31PM  240680        12                         Hush Hush   
7  Jun 28 2022  4:14PM  240679        21                 NBTY Global, Inc.   
8  Jun 28 2022  3:32PM  240672        19               AdvaGen Pharma, Ltd   
9  Jun 28 2022  3:31PM  240673        20  Sartorius Stedim North America     

  Completed Executing Released  
0                            3  
1                            6  
2                   1           
3                            6  
4                            3  
5                            1  
6                            2  
7                            1  
8                           32  
9                            1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                          Customer  \
0  Jun 29 2022  8:21AM  240686        10                 ISDIN Corporation   
1  Jun 29 2022  8:19AM  240688        10                 ISDIN Corporation   
2  Jun 29 2022  8:14AM  240687        16  Sartorius Stedim North America     
3  Jun 29 2022  8:02AM  240685        10                 ISDIN Corporation   
4  Jun 29 2022  8:02AM  240684        10                 ISDIN Corporation   
5  Jun 28 2022  4:38PM  240682        12  Twinlab Consolidated Corporation   
6  Jun 28 2022  4:31PM  240680        12                         Hush Hush   
7  Jun 28 2022  4:14PM  240679        21                 NBTY Global, Inc.   
8  Jun 28 2022  3:32PM  240672        19               AdvaGen Pharma, Ltd   
9  Jun 28 2022  3:31PM  240673        20  Sartorius Stedim North America     

  Released Executing Completed  
0        3                      
1        6                      
2                  1            
3        6                      
4        3                      
5        1                      
6        2                      
7        1                      
8       32                      
9        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                          Customer  \
0  Jun 29 2022  8:21AM  240686        10                 ISDIN Corporation   
1  Jun 29 2022  8:19AM  240688        10                 ISDIN Corporation   
2  Jun 29 2022  8:14AM  240687        16  Sartorius Stedim North America     
3  Jun 29 2022  8:02AM  240685        10                 ISDIN Corporation   
4  Jun 29 2022  8:02AM  240684        10                 ISDIN Corporation   

  Released Executing Completed  
0        3                      
1        6                      
2                  1            
3        6                      
4        3

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                          Customer  \
0  Jun 29 2022  8:21AM  240686        10                 ISDIN Corporation   
1  Jun 29 2022  8:19AM  240688        10                 ISDIN Corporation   
2  Jun 29 2022  8:14AM  240687        16  Sartorius Stedim North America     
3  Jun 29 2022  8:02AM  240685        10                 ISDIN Corporation   
4  Jun 29 2022  8:02AM  240684        10                 ISDIN Corporation   

   Released  Executing  Completed  
0       3.0        NaN        NaN  
1       6.0        NaN        NaN  
2       NaN        1.0        NaN  
3       6.0        NaN        NaN  
4       3.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                          Customer  \
0  Jun 29 2022  8:21AM  240686        10                 ISDIN Corporation   
1  Jun 29 2022  8:19AM  240688        10                 ISDIN Corporation   
2  Jun 29 2022  8:14AM  240687        16  Sartorius Stedim North America     
3  Jun 29 2022  8:02AM  240685        10                 ISDIN Corporation   
4  Jun 29 2022  8:02AM  240684        10                 ISDIN Corporation   

   Released  Executing  Completed  
0       3.0        0.0        0.0  
1       6.0        0.0        0.0  
2       0.0        1.0        0.0  
3       6.0        0.0        0.0  
4       3.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1203414      19.0        0.0        0.0
12          481362       3.0        0.0        0.0
16          240687       0.0        1.0        0.0
18          481320       2.0        0.0        0.0
19          481285      33.0        0.0        0.0
20         1684487      37.0       10.0        7.0
21          721948       3.0        0.0        0.0
22          721932       1.0        2.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1203414      19.0        0.0        0.0
1        12   481362       3.0        0.0        0.0
2        16   240687       0.0        1.0        0.0
3        18   481320       2.0        0.0        0.0
4        19   481285      33.0        0.0        0.0
5        20  1684487      37.0       10.0        7.0
6        21   721948       3.0        0.0        0.0
7        22   721932       1.0        2.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      19.0        0.0        0.0
1        12       3.0        0.0        0.0
2        16       0.0        1.0        0.0
3        18       2.0        0.0        0.0
4        19      33.0        0.0        0.0
5        20      37.0       10.0        7.0
6        21       3.0        0.0        0.0
7        22       1.0        2.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   19.0
1        12  Released    3.0
2        16  Released    0.0
3        18  Released    2.0
4        19  Released   33.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12   16   18    19    20   21   22
Status                                              
Completed   0.0  0.0  0.0  0.0   0.0   7.0  0.0  0.0
Executing   0.0  0.0  1.0  0.0   0.0  10.0  0.0  2.0
Released   19.0  3.0  0.0  2.0  33.0  37.0  3.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12   16   18    19    20   21   22
0          Completed   0.0  0.0  0.0  0.0   0.0   7.0  0.0  0.0
1          Executing   0.0  0.0  1.0  0.0   0.0  10.0  0.0  2.0
2           Released  19.0  3.0  0.0  2.0  33.0  37.0  3.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12   16   18    19    20   21   22
0  Completed   0.0  0.0  0.0  0.0   0.0   7.0  0.0  0.0
1  Executing   0.0  0.0  1.0  0.0   0.0  10.0  0.0  2.0
2   Released  19.0  3.0  0.0  2.0  33.0  37.0  3.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()